# GUIDED EXERCISE 9: Functors and applicatives

## Functors
Functors are the types over which we can map a function. The `Functor` typeclass defines the `fmap` method. Examples of functors are lists or `Maybe`.

In [1]:
-- For lists fmap = map
fmap even [1, 2, 3, 4]
-- Also for Maybe
fmap even (Just 3)
fmap even Nothing

[False,True,False,True]

Just False

Nothing

 We can create types that are functors by implementing the `fmap` function

In [2]:
-- Notice that the type must take one and only one parameter
-- You cannot derive from Functor if you have no parameters or more than one
data Test a = Const1 a | Const2 a | Const3 deriving (Show)
instance Functor Test where
    fmap f (Const1 x) = Const1 (f x)
    fmap f (Const2 x) = Const2 (f x)
    fmap f Const3 = Const3 

In [3]:
-- Applying some functions to the new type
fmap (+1) (Const1 33)
fmap (+1) Const3
fmap even (Const2 3)
fmap even Const3
-- Checking types
:t fmap (+1) (Const1 33)
:t fmap (+1) Const3
:t fmap even (Const2 3)
:t fmap even Const3
-- Notice the type: it takes a function a -> b and a Functor f and returns a Functor
:t fmap

Const1 34

Const3

Const2 False

Const3

fmap (+1) (Const1 33) :: forall b. Num b => Test b

fmap (+1) Const3 :: forall b. Num b => Test b

fmap even (Const2 3) :: Test Bool

fmap even Const3 :: Test Bool

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

I/O actions are also functors so we can use `fmap` with them

In [4]:
-- Function that checks if the input is even
evenInput :: IO ()
evenInput = do
            putStrLn "Enter a number"
            sn <- getLine
            putStrLn ("You entered " ++ sn)
            -- The :: Int is not really needed, it can de infered given that even takes integers
            let n = read sn :: Int
            if even n then putStrLn "Even" else putStrLn "Odd"

-- Same function with fmap
evenInput' :: IO ()
evenInput' = do
            putStrLn "Enter a number"
            n <- fmap read getLine
            putStrLn ("You entered " ++ show n)
            if even n then putStrLn "Even" else putStrLn "Odd"

In [5]:
evenInput'

Enter a number
You entered 55
Odd

**Exercise 1.** Create a function `reverseUpperLine ::  IO()` that reads a line from the keyboard, converts it to upper case and reverses it

In [6]:
capitalize::Char -> Char
capitalize a 
    | null result = a
    | otherwise = head result
    where
        result = [a' | (b, a') <- zip ['a'..'z'] ['A'..'Z'], a == b]

reverseUpperLine ::  IO()
reverseUpperLine = do
    putStrLn "Enter some text"
    text <- fmap (reverse . map capitalize) getLine
    putStrLn $ "You entered " ++ text

In [7]:
reverseUpperLine 

Enter some text
You entered UOY ERA WOH OLLEH

## Applicative functors

The `Applicative` typeclass generalizes the `Functor` typeclass for functions that take more than one parameter. It defines two methods `pure` and `<*>` without any default implementation.

If `f` is a binary function, and `x` and `y` are functors, To apply `f x y` we do `pure f <*> x <*> y` 

In [8]:
-- Add 4 to all elements of a list
pure (+) <*> [1, 2, 3] <*> pure 4
-- Equivalent 1
pure (+) <*> [1, 2, 3] <*> [4]
-- Equivalent 2
map (+4) [1, 2, 3]

Line 2: Use <$>
Found:
pure (+) <*> [1, 2, 3]
Why not:
((+) <$> [1, 2, 3])Line 4: Use <$>
Found:
pure (+) <*> [1, 2, 3]
Why not:
((+) <$> [1, 2, 3])

[5,6,7]

[5,6,7]

[5,6,7]

In [9]:
-- Example: making our Test type to be an Applicative
instance Applicative Test where
    pure = Const1
    (Const1 x) <*> a = fmap x a
    (Const2 x) <*> a = fmap x a
    Const3 <*> _ = Const3

In [10]:
pure (+) <*> Const1 3 <*> Const1 4
pure (+) <*> Const2 3 <*> Const2 5
-- The type is Const1 as: f (Const1 x) = Const1 (f x)
pure (+) <*> Const2 3 <*> Const1 6
pure (+) <*> Const1 3 <*> Const2 7
pure (+) <*> Const3 <*> Const1 7
pure (+) <*> Const1 4 <*> Const3

Line 1: Use <$>
Found:
pure (+) <*> Const1 3
Why not:
((+) <$> Const1 3)Line 2: Use <$>
Found:
pure (+) <*> Const2 3
Why not:
((+) <$> Const2 3)Line 4: Use <$>
Found:
pure (+) <*> Const2 3
Why not:
((+) <$> Const2 3)Line 5: Use <$>
Found:
pure (+) <*> Const1 3
Why not:
((+) <$> Const1 3)Line 6: Use <$>
Found:
pure (+) <*> Const3
Why not:
((+) <$> Const3)Line 7: Use <$>
Found:
pure (+) <*> Const1 4
Why not:
((+) <$> Const1 4)

Const1 7

Const2 8

Const1 9

Const2 10

Const3

Const3

It has to be noticed that `pure f <*> x` equals `fmap f x`. We can substitute the `pure f <*>` by `f <$>` so if we have a function `f a b c` that works for example wiht integers, we can use it with functors by doing: `f <$> a <*> b <*> c`

In [17]:
-- All of these are equivalent
pure (+4) <*> Const2 5
fmap (+4) (Const2 5)
(+4) <$> Const2 5
(+) <$> Const2 5 <*> pure 4
-- Adding two tests, notice the difference in notation with previous cell
(+) <$> Const1 4 <*> Const3

Line 2: Use <$>
Found:
pure (+ 4) <*> Const2 5
Why not:
(+ 4) <$> Const2 5

Const2 9

Const2 9

Const2 9

Const1 9

Const3

**Exercise 2.** Create a function `add x y z ` that adds three elements of the `Test` class

In [12]:
-- Dificult to read way
add :: Num a => Test a -> Test a -> Test a -> Test a
add x y z = (+) <$> ((+) <$> x <*> y) <*> z

-- Easy to read way, with an auxiliary function
add3Numbers :: Num a => a -> a -> a -> a
add3Numbers x y z = x + y + z

add3 :: Num a => Test a -> Test a -> Test a -> Test a
add3 x y z = add3Numbers <$> x <*> y <*> z

In [13]:
-- First implementation
add (Const2 3) (Const2 4) (Const2 6)
add (Const2 3) (Const2 4) (Const1 6)
add (Const2 3) (Const2 4) Const3

-- Second implementation
add3 (Const2 3) (Const2 4) (Const2 6)
add3 (Const2 3) (Const2 4) (Const1 6)
add3 (Const2 3) (Const2 4) Const3

Const2 13

Const1 13

Const3

Const2 13

Const1 13

Const3

**Exercise 3.** Make the previous function generic for any `Applicative` type whose parameter is a `Number`

In [14]:
add3' :: (Applicative f, Num a) => f a -> f a -> f a -> f a
add3' x y z = add3Numbers <$> x <*> y <*> z

In [20]:
add3' (Just 3.0) (Just 1.0) (Just 4.0)
add3' [1] [2] [3]
add3' [1, 2] [2] [3]
add3' [1, 2] [2, 3] [3]

Just 8.0

[6]

[6,7]

[6,7,7,8]